# 🙀 Cat Evaluation

### 📝 Imports

In [1]:
import numpy

import torch
import torch.optim as optim

from torch.utils.data import DataLoader

import albumentations as A

import torchvision.transforms as transforms

import os

from src.cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from src.augmentation.data_augmenter import DataAugmenter

from src.cats_dataset import CatsDataset

c:\Users\Dan\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 🔧 Config

In [2]:
image_size = 512

saved_model_path = "trained_networks/cat_discriminator.pth"

testing_data_path = 'data/test'

### 🌐 Create Transforms

In [3]:
transform = transforms.Compose([
    DataAugmenter(target_image_size=image_size, augment_images=False),
    transforms.ToTensor(), # converts numpy to trans
])

### 🤓📝 Load Testing Data

In [4]:
dataset = CatsDataset(
    root_dir=testing_data_path,
    transform=transform)

data_loader = DataLoader(dataset, shuffle=False, num_workers=4)

### 🥾 Initialize the Neural Net

In [5]:
net = CatDiscriminatorNeuralNet(saved_model_path=saved_model_path)
net.cuda();

TypeError: CatDiscriminatorNeuralNet.__init__() missing 1 required positional argument: 'learning_rate'

### 🤖 Evaluate

In [6]:
evaluation_result = net.evaluate(data_loader=data_loader)
evaluation_result.print_verbose_results()

Number correct: 272
Total evaluations: 300
Percent correct: 90.66666666666666%


Actual: bathroom-cat, Predicted: bathroom-cat
bathroom-cat: 7.3564, captain: -2.4858, control: -5.7487
bathroom-cat: 99.9945%, captain: 0.0053%, control: 0.0002%

Actual: bathroom-cat, Predicted: bathroom-cat
bathroom-cat: 5.5279, captain: -2.4780, control: -3.6658
bathroom-cat: 99.9565%, captain: 0.0333%, control: 0.0102%

Actual: bathroom-cat, Predicted: bathroom-cat
bathroom-cat: 11.8674, captain: -3.4042, control: -9.9467
bathroom-cat: 100.0000%, captain: 0.0000%, control: 0.0000%

Actual: bathroom-cat, Predicted: captain
bathroom-cat: 1.0679, captain: 1.6473, control: -2.4949
bathroom-cat: 35.5455%, captain: 63.4464%, control: 1.0081%

Actual: bathroom-cat, Predicted: bathroom-cat
bathroom-cat: 3.1604, captain: -0.9345, control: -2.4478
bathroom-cat: 98.0080%, captain: 1.6326%, control: 0.3594%

Actual: bathroom-cat, Predicted: bathroom-cat
bathroom-cat: 9.1656, captain: -3.5882, control: -6.7474
bath

### 💾 Save Progress

In [7]:
torch.save(net.state_dict(), saved_model_path)